In [4]:
import sys
import os
import ee
import geemap
import rasterio
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as colors

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'Enschede_Farmland.tif')
filename_corine = os.path.join(out_dir, 'Enschede_Farmland_Corine.tif')

try:
    # Initialize the library.
    ee.Authenticate()
    ee.Initialize()
    print('Google Earth Engine has initialized successfully!')
except ee.EEException as e:
    print('Google Earth Engine has failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise



Successfully saved authorization token.
Google Earth Engine has initialized successfully!


In [10]:
#define region to look at
region = ee.Geometry.Polygon([[[6.75627, 52.133732],
  [6.85618, 52.133732],
  [6.85618, 52.170806],
  [6.75627, 52.170806],
  [6.75627, 52.133732]]]
)

In [5]:
# Some visualization parameters:
# - Values to map to 0 and values to map to 255.
# - Three bands to be combined as an rgb image

# You can try changing these parameters and check the result!

vis_params = {"min": 0,
              "max": 1500,
              "bands": ["B4", "B3", "B2"]}
vis_params_corine = {"min": 0,
              "max": 1500}


In [30]:
img1corine = ee.Image('COPERNICUS/CORINE/V20/100m/2018') \
                  .select('landcover')
#                   .clip(region) \
region = ee.Geometry.Polygon([[[6.703071, 52.183918],
  [6.943405, 52.183918],
  [6.943405, 52.32557],
  [6.703071, 52.32557],
  [6.703071, 52.183918]]])

# Select only the pixels that belong to the farmland classes (211 and 242)
farmlandMask = img1corine.eq(211) \
                .Or(img1corine.eq(212)) \
                .Or(img1corine.eq(213)) \
                .Or(img1corine.eq(221)) \
                .Or(img1corine.eq(222)) \
                .Or(img1corine.eq(223)) \
                .Or(img1corine.eq(231)) \
                .Or(img1corine.eq(241)) \
                .Or(img1corine.eq(242)) \
                .Or(img1corine.eq(243)) \
                .Or(img1corine.eq(244))\
                .clip(region) 



Map2 = geemap.Map(location=[52.191838, 6.741434], zoom=13)
Map2.addLayer(farmlandMask, {}, "FirstImage", True)
Map2

Map(center=[52.191838, 6.741434], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

In [43]:
n_clusters = 2

training = farmlandMask.sample(**{
  "scale": 30,
  "numPixels": 5000
})

clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)
clusteredFc = training.cluster(clusterer, 'spectral_cluster');
palette = ee.List(['8dd3c7', 'ffffb3', 'bebada', 'fb8072', '80b1d3']);



TypeError: 'Image' object is not callable

In [49]:
result = farmlandMask.cluster(clusterer)

# # Display the clusters with random colors.
Map = geemap.Map(location=[52.191838, 6.741434], zoom=13)
Map.addLayer(result.randomVisualizer(), {'min': 0, 'max' : 10}, 'clusters')
Map


Map(center=[52.191838, 6.741434], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…